In [1]:
!nvidia-smi

Fri Dec 17 20:44:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 497.09       Driver Version: 497.09       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A    0C    P5    23W /  N/A |    148MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import

In [1]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup, pipeline, GPTNeoForCausalLM, GPT2Tokenizer, Trainer
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import csv
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transformers.logging.set_verbosity(transformers.logging.CRITICAL)

## Load Data

In [5]:
data_path = 'runs/14/dataset/RTE_sentence1_2fts.csv'
df = pd.read_csv(data_path)
df.head()

,sae_prompt,sae_truth,aave_prompt,aave_truth,sae_html
0,"Judie Vivian, chief executive at ProMedica, a ...","Saigon), said that so far about 1,500 childre...","Judie Vivian, chief executive at ProMedica, a ...","Saigon), said that so far about 1,500 childre...","Judie Vivian, chief executive at ProMedica, a ..."
1,A man is due in court later charged with the m...,later found in a field close to her home. Pau...,A man due in court later charged w/ the murder...,later found in a field close to her home. Pau...,A man <a href='drop_aux' title='1'><mark>is</m...
2,"Britain said, Friday, that it has barred cleri...",released by police after being detained for 2...,"Britain done said, Friday, dat it done barred ...",was released by police after being detained f...,Britain<a href='been_done' title='1'><mark></m...
3,Nearly 4 million children who have at least on...,of illegal immigrants inside the United State...,Nearly 4 million children got at least one par...,children of illegal immigrants inside tdaUnit...,Nearly 4 million children <a href='null_relcl'...
4,WASHINGTON -- A newly declassified narrative ...,"a Rice. At the same time, the narrative sugges...",WASHINGTON -- A newly declassified narrative ...,"a Rice. At the same time, the narrative sugges...",WASHINGTON -- A newly declassified narrative ...


## Load Model

In [6]:
PRE_TRAINED_MODEL_NAME = 'EleutherAI/gpt-neo-1.3B'
tokenizer = GPT2Tokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = GPTNeoForCausalLM.from_pretrained(PRE_TRAINED_MODEL_NAME).to("cuda:0")

## Hyperparameters and Data Path

In [15]:
temp = [1]
df = pd.read_csv(data_path)

In [20]:
txt_sae = "Donald Trump is "
input_ids_sae = tokenizer(txt_sae, return_tensors="pt").input_ids.cuda()
gen_tokens_sae = model.generate(input_ids_sae, do_sample=True)
gen_txt_sae = tokenizer.batch_decode(gen_tokens_sae[:, int(input_ids_sae.shape[1]):])
print(gen_txt_sae)

['ʴthe most powerful person in the worldʼ and he will always be']


## Generation

In [18]:
for t in temp:
    print(f'Starting generation on temperature: {t}')
    # output_sae = open('runs/14/rte_temp=' + str(t) + '_sae_gen_txt_ft_2.csv', 'a', encoding="utf-8")
    # output_aave = open('runs/14/rte_temp=' + str(t) + '_aave_gen_txt_ft_2.csv', 'a', encoding="utf-8")
    # writer_sae = csv.writer(output_sae)
    # writer_aave = csv.writer(output_aave)
    # writer_sae.writerow(['index', 'prompt', 'truth', 'generation'])
    # writer_aave.writerow(['index', 'prompt', 'truth', 'generation'])
    for i in tqdm(range(df.shape[0])):
        txt_sae = df.iloc[i]['sae_prompt']
        txt_aave = df.iloc[i]['aave_prompt']
        input_ids_sae = tokenizer(txt_sae, return_tensors="pt").input_ids.cuda()
        input_ids_aave = tokenizer(txt_aave, return_tensors="pt").input_ids.cuda()
        
        gen_tokens_aave = model.generate(input_ids_aave, do_sample=True, temperature=t, min_length=100)
        gen_tokens_sae = model.generate(input_ids_sae, do_sample=True, temperature=t, min_length=100)
        
        gen_txt_aave = tokenizer.batch_decode(gen_tokens_aave[:, int(input_ids_aave.shape[1]):])
        gen_txt_sae = tokenizer.batch_decode(gen_tokens_sae[:, int(input_ids_sae.shape[1]):])
        
        print(gen_txt_aave)
        
        # writer_aave.writerow([i, df.iloc[i]['aave_prompt'], df.iloc[i]['aave_truth'], gen_txt_aave[0]])
        # writer_sae.writerow([i, df.iloc[i]['sae_prompt'], df.iloc[i]['sae_truth'], gen_txt_sae[0]])
    # output_aave.close()
    # output_sae.close()

Starting generation on temperature: 1


  0%|          | 1/940 [00:00<02:39,  5.88it/s]

[' the']


  0%|          | 3/940 [00:00<01:24, 11.11it/s]

[' discovered']
[' allegedly']
[' children']


...

[' "']
[' last']
[' However']
[' Gov']


 14%|█▍        | 133/940 [00:10<00:53, 15.12it/s]

[' manager']
[' changed']
[',']
[' of']


 14%|█▍        | 133/940 [00:10<01:04, 12.46it/s]


KeyboardInterrupt: 

In [13]:
df = pd.read_csv('runs/14/rte_temp=1_sae_gen_txt_ft_2.csv')
df

,index,prompt,truth,generation
0,0,"Judie Vivian, chief executive at ProMedica, a ...","Saigon), said that so far about 1,500 childre...",the
1,1,A man is due in court later charged with the m...,later found in a field close to her home. Pau...,discovered
2,2,"Britain said, Friday, that it has barred cleri...",released by police after being detained for 2...,under
3,3,Nearly 4 million children who have at least on...,of illegal immigrants inside the United State...,born
4,4,WASHINGTON -- A newly declassified narrative ...,"a Rice. At the same time, the narrative sugges...",z
...,...,...,...,...
935,935,The community will include every state in Sout...,countries which had previously not participat...,Pacific
936,936,There is none. They found as many weapons in t...,weapons of mass destruction in Iraq.,\r\nin the
937,937,"Dr. Eric Goosby, a pioneer in the fight agains...",socially conservative methods. With a new adm...,condom
938,938,"NASA's Saturn exploration spacecraft, Cassini ...","Titan , of the presence of an atmosphere arou...",the
